# Contexto 🤖

Notebook voltado ao escopo do A. Supervisionado relacionado à classificação. Trata-se de uma revisão acerca desse tipo de aprendizado, construído junto de um estudo de caso referente à análise de um conjunto de dados que relaciona características de seus clientes com a sua respectiva aderência, ou falta, ao investimento.

>

O notebook é didivido no primeiro momento para a análise da condição dos dados, verificando se possuem valores duplicados ou faltantes, sendo seguido por uma etapa de exploração nos dados, buscando relações e insights, chegando na criação dos modelos de machine learning (ML), os quais irão treinar e, posteriormente, prever as classes dos clientes, como forma de saber quais, com base em suas características, podem ser mais ou menos aderentes ao investimento.

>

Por fim, há a etapa da validação dos modelos criados, os quais serão comparados segundo determinadas métricas, como o score, f1-score e precisão.

## Legenda - Marketing e Investimento :

- idade: Idade do cliente em anos.


- estado_civil: Estado civil do cliente, categorizado como "casado (a)", "solteiro (a)", "divorciado (a)".


- escolaridade: Nível de escolaridade do cliente, categorizado como
"fundamental", "médio", "superior".

- inadimplencia: Indica se o cliente possui alguma inadimplência financeira, categorizado como "sim" ou "não".

- saldo: Saldo financeiro do cliente.

- fez_emprestimo: Indica se o cliente já fez algum empréstimo,
categorizado como "sim" ou "não".

- tempo_ult_contato: Tempo decorrido, em dias, desde o último contato com o cliente.

- numero_contatos: Número de vezes que o cliente foi contatado.

- aderencia_investimento: Indica se o cliente aderiu a algum

- investimento oferecido, categorizado como "sim" ou "não".

## Importando as bibliotecas 📚

In [403]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix



In [326]:
# Armazenando o dataset :

file_path = '/content/marketing_investimento.csv'

df = pd.read_csv(file_path)

## Realizando a primeira exploração nos dados 🔍



In [327]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   idade                   1268 non-null   int64 
 1   estado_civil            1268 non-null   object
 2   escolaridade            1268 non-null   object
 3   inadimplencia           1268 non-null   object
 4   saldo                   1268 non-null   int64 
 5   fez_emprestimo          1268 non-null   object
 6   tempo_ult_contato       1268 non-null   int64 
 7   numero_contatos         1268 non-null   int64 
 8   aderencia_investimento  1268 non-null   object
dtypes: int64(4), object(5)
memory usage: 89.3+ KB


In [328]:
df.duplicated().sum()

0

In [329]:
df.isnull().sum()

idade                     0
estado_civil              0
escolaridade              0
inadimplencia             0
saldo                     0
fez_emprestimo            0
tempo_ult_contato         0
numero_contatos           0
aderencia_investimento    0
dtype: int64

In [330]:
df.isna().sum()

idade                     0
estado_civil              0
escolaridade              0
inadimplencia             0
saldo                     0
fez_emprestimo            0
tempo_ult_contato         0
numero_contatos           0
aderencia_investimento    0
dtype: int64

In [331]:
df.head()

,idade,estado_civil,escolaridade,inadimplencia,saldo,fez_emprestimo,tempo_ult_contato,numero_contatos,aderencia_investimento
0,45,casado (a),superior,nao,242,nao,587,1,sim
1,42,casado (a),medio,nao,1289,nao,250,4,sim
2,23,solteiro (a),superior,nao,363,nao,16,18,nao
3,58,divorciado (a),superior,nao,1382,nao,700,1,sim
4,50,casado (a),medio,nao,3357,nao,239,4,sim


In [332]:
df.shape

(1268, 9)

## Analisando o nível de outliers nos dados

In [379]:
for col in df.columns:
    fig = px.box(df, y=col,
                 title=f'Boxplot de {col}',
                 template='plotly_dark',
                 points='outliers', # Mostra apenas os outliers
                 color_discrete_sequence=['darkgray'],
                 width=500,  # Largura em pixels
                 height=400  # Altura em pixels
                 )
    fig.show()

Analisando os gráficos de boxplot, concebe-se que os dados apresentam significativa presença de outliers, que podem prejudicar a análise dos modelos de ML futuramente criados. Desse modo, concebe-se que para a mitigação desse efeito, os dados precisarão ser tratados, quer seja mediante a uma normalização ou transformação, bem como recomenda-se que sejam analisados por modelos que apresentam consistência a outliers, como os modelos de árvores de decisão e random forest.

>

Os dados que apresentam os outliers são :     

- numero_contatos
- tempo_ult_contato
- saldo
- idade

## Analisando a relação idade x aderência ao investimento 🔬

In [334]:
# Agrupamento dos dados por idade e cálculo da porcentagem de aderência
df_grouped = df.groupby('idade')['aderencia_investimento'].value_counts(normalize=True).unstack()
df_grouped *= 100

# Criação do gráfico de barras
fig = px.bar(df_grouped,
             barmode='group',
             labels={'value': 'Porcentagem', 'idade': 'Idade', 'variable': 'Aderência a Investimento'},
             title='Relação entre Idade e Aderência a Investimento',
             color_discrete_map={'sim': 'darkgreen', 'nao': 'darkred'},
             template='plotly_dark')

fig.show()

Percentualmente, por meio desse gráfico, compreende-se que a aderência ao investimento é menos prevalente nos grupos que estão num intervalo entre 30 a 60 anos, ainda que em certas faixas a relação de aderência e não aderência sejam proporcionais.

In [335]:
# Criar o gráfico de distribuição para 'aderencia_investimento' == 'sim'
fig_sim = px.histogram(df[df['aderencia_investimento'] == 'sim'],
                       x='idade',
                       nbins=20,
                       title='Distribuição de Idade para Aderência "Sim"',
                       labels={'idade': 'Idade'},
                       color_discrete_sequence=['darkgreen'],
                       template='plotly_dark')

fig_sim.show()

In [336]:
# Criar o gráfico de distribuição para 'aderencia_investimento' == 'nao'
fig_nao = px.histogram(df[df['aderencia_investimento'] == 'nao'],
                       x='idade',
                       nbins=20,
                       title='Distribuição de Idade para Aderência "Não"',
                       labels={'idade': 'Idade'},
                       color_discrete_sequence=['darkred'],
                       template='plotly_dark')

fig_nao.show()

Por meio desses gráficos, nota-se que a idade não se coloca, no presente dataset, com uma principal determinante na maior ou menor aderência ao investimento, com uma pequena exceção no que se refere à quantidade de investimento não contraídos conforme a idade avança, podendo ser percebido no intervalo de 54 a 60 anos.

In [337]:
# Verificando os dados estatísticos referentes
# aos clientes e o nível de aderência ao investimento.

# Para os são aderentes

df[df['aderencia_investimento'] == 'sim'].describe()

,idade,saldo,tempo_ult_contato,numero_contatos
count,502.000000,502.000000,502.000000,502.000000
mean,42.163347,1541.430279,556.942231,2.256972
std,13.096190,2435.044454,392.352339,2.011866
min,19.000000,-1206.000000,30.000000,1.000000
25%,32.000000,160.500000,261.250000,1.000000
50%,39.000000,700.000000,452.500000,2.000000
75%,50.000000,2130.250000,758.500000,3.000000
max,87.000000,26965.000000,2769.000000,24.000000


In [338]:
df[df['aderencia_investimento'] == 'nao'].describe()

,idade,saldo,tempo_ult_contato,numero_contatos
count,766.000000,766.000000,766.000000,766.000000
mean,40.861619,1379.199739,243.472585,3.015666
std,10.081455,2845.080617,246.514996,3.303358
min,22.000000,-921.000000,5.000000,1.000000
25%,33.000000,61.250000,99.500000,1.000000
50%,39.000000,411.500000,179.000000,2.000000
75%,48.000000,1403.250000,297.750000,3.000000
max,83.000000,27069.000000,3025.000000,32.000000


Analisando os dados estatísticos referentes a aderência ou não aderência ao investimento.

## Analisando a relação de escolaridade e aderência ao investimento 🔬

In [339]:
# Contagem da aderência por escolaridade
df_escolaridade = df.groupby('escolaridade')['aderencia_investimento'].value_counts().unstack()

# Criação do gráfico de barras
fig = px.bar(df_escolaridade,
             barmode='group',
             labels={'value': 'Quantidade', 'escolaridade': 'Escolaridade', 'variable': 'Aderência a Investimento'},
             title='Relação entre Escolaridade e Aderência a Investimento',
             color_discrete_map={'sim': 'darkgreen', 'nao': 'darkred'},
             template='plotly_dark')

fig.show()

Gráfico que concebe a relação sobre a quantidade de aderência ao investimento com base na escolaridade dos clientes presentes no dataframe.

Por meio dele é possível perceber que proporcionalmente há mais pessoas que não são aderentes aos investimentos em relação das que são, de modo que, em cada nível de escolaridade, os que não aderem, percentualmente, são superiores em relação aos que aderem.

Entretanto, apesar de manter esse padrão, a menor diferença dos que aderem em relação aos que não aderem se dá no grupo dos clientes que possuem escolaridade superior.

Não obstante, a escolaridade de nível médio é o grupo que encontra a maior quantidade de dados, em ambos os casos, o que pode indicar que a maior quantidade dos clientes presentes no dataset possuem escolaridade até o ensino médio.

## Analisando escolaridade, saldo e aderência ao investimento



In [340]:
fig = px.scatter(df,
                 x='escolaridade',
                 y='saldo',
                 color='aderencia_investimento',
                 labels={'escolaridade': 'Escolaridade', 'saldo': 'Saldo', 'aderencia_investimento': 'Aderência a Investimento'},
                 title='Relação entre Escolaridade, Saldo e Aderência a Investimento',
                 color_discrete_map={'sim': 'darkgreen', 'nao': 'darkred'},
                 template='plotly_dark')

fig.show()

Por meio desse gráfico, com o dataset disponível e os dados dos clientes não é possível perceber uma relação clara entre o nível da escolaridade, saldo e aquisição de investimento, no que se refere àqueles que possuem nível superior contrair investimentos em quantidade significativa em relação aos demais. A diferença só é clara para aqueles que possuem apenas ensino fundamental, sendo pequena para os que possuem nível médio.



## Analisando idadimplência e escolaridade

In [341]:
# Contagem de inadimplência por escolaridade
df_escolaridade = df.groupby('escolaridade')['inadimplencia'].value_counts().unstack().fillna(0)

# Criação do gráfico de barras
fig = px.bar(df_escolaridade,
             barmode='group',
             labels={'value': 'Quantidade', 'escolaridade': 'Escolaridade', 'variable': 'Inadimplência'},
             title='Relação entre Escolaridade e Inadimplência',
             color_discrete_map={'sim': 'darkred', 'nao': 'darkgreen'},
             template='plotly_dark')

fig.show()

In [342]:
df.inadimplencia.value_counts()

inadimplencia
nao    1245
sim      23
Name: count, dtype: int64

Tanto visualmente quanto numericamente, constata-se que os clientes desse banco tendem a não possuir inadimplência, tendo em vista que o percentual dos que possuem é ínfimo em comparação com os que não possuem.

Entretanto, concebe-se que dos que são inadimplentes, aqueles que possuem apenas o ensino médio é superior em relação aos demais, sendo seguido pelos que possuem ensino superiore fundamental, respectivamente.



## Analisando a relação entre estado cível, empréstimo e aderência ao investimento

In [343]:
# Criar subplots com 1 linha e 2 colunas
fig = make_subplots(rows=1, cols=2, subplot_titles=("Estado Civil e Empréstimo", "Estado Civil e Aderência a Investimento"))

# Gráfico 1: Estado Civil e Empréstimo
df_emprestimo = df.groupby('estado_civil')['fez_emprestimo'].value_counts().unstack().fillna(0)
fig.add_trace(px.bar(df_emprestimo,
                     barmode='group',
                     labels={'value': 'Quantidade', 'estado_civil': 'Estado Civil', 'variable': 'Fez Empréstimo'},
                     color_discrete_map={'sim': 'darkblue', 'nao': 'darkgray'}).data[0],
              row=1, col=1)

# Gráfico 2: Estado Civil e Aderência a Investimento
df_investimento = df.groupby('estado_civil')['aderencia_investimento'].value_counts().unstack().fillna(0)
fig.add_trace(px.bar(df_investimento,
                     barmode='group',
                     labels={'value': 'Quantidade', 'estado_civil': 'Estado Civil', 'variable': 'Aderência a Investimento'},
                     color_discrete_map={'sim': 'darkgreen', 'nao': 'darkred'}).data[0],
              row=1, col=2)

# Ajustar layout
fig.update_layout(template='plotly_dark', showlegend=False, title_text="Relação entre Estado Civil, Empréstimo e Investimento")
fig.show()

In [344]:
df.estado_civil.value_counts()

estado_civil
casado (a)        738
solteiro (a)      368
divorciado (a)    162
Name: count, dtype: int64

Analisando os gráficos e considerando a proporção de cada grupo em relação ao conjunto de dados, concebe-se que não há uma relação evidente que acerca do estado civil do cliente e a sua aquisição de empréstimos ou de investimento, de modo que a proporção dos dados se mantém semelhantes em ambos os cenários.

## Identificando a correlação entre as colunas do dataframe ( Matriz de correlação )

In [345]:
# Para analisar a matriz de correlação, preciso, antes
# de transformar as variáveis categóricas em numéricas,
# porém mantendo a sua semântica. Para tanto, irei dummerizá-las,
# isto é, atribuir o valor de 1 para aquelas que possuem caráter
# de 'sim' ou pertencer a um grupo e 0 para o caso contrário.

# Selecionar as colunas categóricas
categorical_cols = ['estado_civil', 'escolaridade', 'inadimplencia', 'fez_emprestimo', 'aderencia_investimento']

# Criar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sparse=False para retornar um array denso

# Ajustar e transformar as colunas categóricas
encoded_data = encoder.fit_transform(df[categorical_cols])

# Criar um novo dataframe com as variáveis dummies
df_dummy = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Concatenar o dataframe original com as variáveis dummies
df_dummy = pd.concat([df.drop(categorical_cols, axis=1), df_dummy], axis=1)

In [346]:
df_dummy.head()

,idade,saldo,tempo_ult_contato,numero_contatos,estado_civil_casado (a),estado_civil_divorciado (a),estado_civil_solteiro (a),escolaridade_fundamental,escolaridade_medio,escolaridade_superior,inadimplencia_nao,inadimplencia_sim,fez_emprestimo_nao,fez_emprestimo_sim,aderencia_investimento_nao,aderencia_investimento_sim
0,45,242,587,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,42,1289,250,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,23,363,16,18,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,58,1382,700,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,50,3357,239,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [347]:
# Calcular a matriz de correlação
correlation_matrix = df_dummy.corr()

# Criar o heatmap com valores de correlação
fig = px.imshow(correlation_matrix,
                labels=dict(x="Variáveis", y="Variáveis", color="Correlação"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='RdBu',  # Escala de cores vermelho-azul
                template='plotly_dark',
                title='Mapa de Calor de Correlações',
                text_auto=True,  # Adicionar valores de correlação no gráfico
                aspect="auto"  # Ajustar a proporção do gráfico
                )

# Ajustar a resolução
fig.update_layout(width=1500, height=1000)

fig.show()

Analisando a matriz de correlação, concebe-se que as colunas que apresentaram mais relação positiva de o cliente ser mais aderente ao investimento foram : tempo último de contato, escolaridade superior e se fez não fez empréstimo antes.Por outro lado, os que apresentam relação negativa são número de contatos e se fez empréstimo.      

>

Ou seja, por meio dela pode-se inferir que os clientes que são mais aderentes aos investimentos são aqueles que possuem nível superior, que não fizeram empréstimos anteriores e que foram contactados recentemente pelo banco.

>

Em contrapartida, clientes que foram bastante contactados e que fizeram empréstimos possuem uma propensão maior a não aderirem ao investimento.

## Separando os dados em X e y :    

In [348]:
# Criar o LabelEncoder
label_encoder = LabelEncoder()

# Ajustar o encoder aos dados da coluna 'aderencia_investimento' e transformar a coluna
df['aderencia_investimento'] = label_encoder.fit_transform(df['aderencia_investimento'])

In [349]:
y = df['aderencia_investimento']

In [350]:
X = df_dummy.drop(['aderencia_investimento_nao', 'aderencia_investimento_sim'], axis = 1)

In [351]:
X.head()

,idade,saldo,tempo_ult_contato,numero_contatos,estado_civil_casado (a),estado_civil_divorciado (a),estado_civil_solteiro (a),escolaridade_fundamental,escolaridade_medio,escolaridade_superior,inadimplencia_nao,inadimplencia_sim,fez_emprestimo_nao,fez_emprestimo_sim
0,45,242,587,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,42,1289,250,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,23,363,16,18,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,58,1382,700,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,50,3357,239,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [352]:
pd.DataFrame(y).head(5)

,aderencia_investimento
0,1
1,1
2,0
3,1
4,1


In [353]:
y.value_counts()

aderencia_investimento
0    766
1    502
Name: count, dtype: int64

## Separando em dados de treino e teste

Na separação de treino e teste, estou utilizando o stratify para estratificar os dados segundo a variável target, como forma de garantir que a proporção dos dados seja mantida. Não obstante, os dados estarão divididos numa proporção de 25% de teste e 75% de treino.
  

In [ ]:
SEED = 22

In [354]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,
                                                    random_state = SEED)

## Criando um modelo de baseline ( Dummy Classifier ) 🤡

O modelo de baseline é útil, pois ele serve de comparação com os demais modelos criados, como forma de avaliar a qualidade desses.

In [355]:
# Instanciando o Dummy Classifier :
dummy = DummyClassifier(random_state = SEED)

# Treinando o modelo :
dummy.fit(X_train, y_train)

# Avaliando o modelo por meio do critério score :
dummy_socore = dummy.score(X_test, y_test).round(3)*100
dummy_socore

60.3

## Criando os modelos de machine learning 🤖

## DecisionTreeClassifier 🌲

In [356]:
# Aqui, por mais que pareça que não estamos passando
# o critério por meio do qual a árvore realiza a sepação
# dos dados, com base nas impurezas, está sendo adotado
# o critério, sendo, no caso, o de Gini, que favorece
# divisões que resultam em nós filhos com maior
# homogeneidade de classes.

# Instanciando o modelo de machine learning :
d_tree = DecisionTreeClassifier(max_leaf_nodes = 500,
                                min_samples_leaf = 10, min_samples_split = 2,
                                random_state = SEED)

# Treinando o modelo :
d_tree.fit(X_train, y_train)

# Avaliando o modelo por meio do critério score :
d_tree_score = d_tree.score(X_test, y_test).round(3)*100
d_tree_score

71.0

## RandomForestClassifier 🌲🌲🌲

In [357]:
# Instanciando o modelo :
r_tree = RandomForestClassifier(n_estimators = 200, max_leaf_nodes = 200,
                                max_features = 'sqrt', min_samples_leaf = 10,
                                min_samples_split = 5, random_state = SEED)

# Treinando o modelo :
r_tree.fit(X_train, y_train)

# Avaliando o modelo por meio do critério score :
r_tree_score = r_tree.score(X_test, y_test).round(3)*100
r_tree_score

76.0

## k-Nearest-Neighbors (KNN) 📏

Modelo de ML baseado na distância entre os dados, em especial das variáveis explicativas em relação à variável resposta, como forma de realizar a sua predição ou classificação. Uma vez que é sensível a dados assimétricos e a outliers, precisa de uma etapa de pré-processamento na qual os dados são sujeitos a uma particular distribuição.

>
       
A distribuição pode se dar sob um intervalo entre valores mínimos, começando a partir do zero, e valores máximos, findando no um, bem como por meio de uma normalização, através da qual coloca-se a média dos dados no valor zero, sujeitos a um determinado desvio padrão - essa técnica é mais robusta a presença de outliers.

>

A distribuição dos dados selecionada for a de normalização.

## Normalizando os dados 📐

In [358]:
X.head()

,idade,saldo,tempo_ult_contato,numero_contatos,estado_civil_casado (a),estado_civil_divorciado (a),estado_civil_solteiro (a),escolaridade_fundamental,escolaridade_medio,escolaridade_superior,inadimplencia_nao,inadimplencia_sim,fez_emprestimo_nao,fez_emprestimo_sim
0,45,242,587,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,42,1289,250,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,23,363,16,18,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,58,1382,700,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,50,3357,239,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [359]:
# Instanciando o normalizador :
scaler = StandardScaler()

# Ajustando e transformando os dados, normalizando-os :
X_normalizado = scaler.fit_transform(X[['idade', 'saldo',
                                        'tempo_ult_contato', 'numero_contatos']])

In [360]:
# Concatenando o dataframe normalizado com o dataframe que
# armazena, compreende as variáveis explicativas do conjunto de dados.

X_normalizado = pd.concat([pd.DataFrame(X_normalizado),
                           X.drop(['idade', 'saldo',
                                   'tempo_ult_contato',
                                   'numero_contatos'], axis = 1)], axis = 1)

In [365]:
# Renomeando as colunas referente
# aos dados que foram normalizados

X_normalizado = X_normalizado.rename(columns = {0 : 'idade', 1 : 'saldo',
                                2 : 'tempo_ult_contato',
                                3 : 'numero_contatos'})

In [366]:
X_normalizado.head()

,idade,saldo,tempo_ult_contato,numero_contatos,estado_civil_casado (a),estado_civil_divorciado (a),estado_civil_solteiro (a),escolaridade_fundamental,escolaridade_medio,escolaridade_superior,inadimplencia_nao,inadimplencia_sim,fez_emprestimo_nao,fez_emprestimo_sim
0,0.318387,-0.446736,0.630836,-0.59466,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,0.054751,-0.057422,-0.338022,0.44538,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,-1.614945,-0.401743,-1.010759,5.29890,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1.460811,-0.022841,0.955705,-0.59466,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0.757781,0.711539,-0.369646,0.44538,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [367]:
# Segmentando novamente em porção de treino e teste,
# segundo a uma estratificação baseada na variável target.

X_train, X_test, y_train, y_test = train_test_split(X_normalizado, y,
                                                    stratify = y,
                                                    random_state = SEED)

In [369]:
# Instanciando o modelo e aplicando a métrica
# de distância Manhattan, a qual é mais robusta
# a presença de outliers.

knn = KNeighborsClassifier(metric = 'manhattan')

# Treinando o modelo
knn.fit(X_train, y_train)

# Avaliando o modelo pelo critério de score :
knn_socore = knn.score(X_test, y_test).round(3)*100
knn_socore

73.2

## Modelo de Regressão Logística 📈

In [375]:
# Instanciando o modelo :
log_regression = LogisticRegression()

# Treinando o modelo :
log_regression.fit(X_train, y_train)

# Avaliando o modelo segundo o critério de score :
log_regression_score = log_regression.score(X_test, y_test).round(3)*100
log_regression_score

76.0

In [382]:
f1_score(y_test, log_regression.predict(X_test)).round(3)

0.667

## Validação 📝

Etapa destinada à validação dos modelos, por meio das métricas de taxa de acerto (acurácia), precisão e f1-socore (a média harmônica entre a precisão e o recall)

In [408]:
def validaModelos(nome_modelo, modelo):

  modelo = modelo
  modelo.fit(X_train, y_train)
  y_predict = modelo.predict(X_test)

  acuracia =  accuracy_score(y_predict, y_test).round(2)*100
  precision = precision_score(y_test, y_predict).round(2)*100
  recall = recall_score(y_test, y_predict).round(2)*100
  f1 = f1_score(y_test, y_predict).round(3)*100

  print(f"\n{nome_modelo}\n")
  print(f'Acurácia : {acuracia} %')
  print(f'Precisão : {precision} %')
  print(f'Recall : {recall} %')
  print(f'F1-score : {f1} % ')


In [409]:
# Lista dos modelos usados :

modelos = [
    ("Dummy Classifier", dummy),
    ("Árvore de Decisão", d_tree),
    ("Random Forest", r_tree),
    ("KNN", knn),
    ("Regressão Logística", log_regression)
]

# Iterando pelos modelos e avaliando
for nome_modelo, modelo in modelos:
    validaModelos(nome_modelo, modelo)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.




Dummy Classifier

Acurácia : 60.0 %
Precisão : 0.0 %
Recall : 0.0 %
F1-score : 0.0 % 

Árvore de Decisão

Acurácia : 71.0 %
Precisão : 68.0 %
Recall : 50.0 %
F1-score : 57.8 % 

Random Forest

Acurácia : 76.0 %
Precisão : 70.0 %
Recall : 69.0 %
F1-score : 69.6 % 

KNN

Acurácia : 73.0 %
Precisão : 68.0 %
Recall : 60.0 %
F1-score : 64.1 % 

Regressão Logística

Acurácia : 76.0 %
Precisão : 75.0 %
Recall : 60.0 %
F1-score : 66.7 % 


Analisando os modelos, concebe-se que o melhor modelo, com base em sua acurácia, precisão e recall é o Random Forest. Por mais que o modelo de regressão logística possui um valor de precisão interessante, o seu recall é significativamente inferior a ela, produzindo uma média harmônica inferior à gerada pelo modelo Random Forest.

## Em síntese...

Portanto, esse notebook permite concluir, com base na análise exploratória dos dados, de que as variáveis positivamente determinantes para a aderência ao investimento do cliente é o seu nível de escolaridade, a não obtenção de empréstimo anterior e intervalo mínimo de contato.

Da mesma forma, o melhor modelo de ML para o present dataset é o de Random Forest, o qual possui tanto uma boa acurácia, quanto média harmônica dos valores de precisão e recall, demonstrando uma boa assertividade acerca da classificaçãos dos clientes, com base nas variáveis explicativas.